In [1]:
import torchio as tio
from nibabel.testing import data_path
import nibabel as nib
import os
from copy import deepcopy
import numpy as np
import timeit

np.random.seed(4215235)
iterations = 2
# We use a publicly available sample from Nibabel
example_file = os.path.join(data_path, 'example4d.nii.gz')
im = nib.load(example_file)
imarr = im.get_fdata()[:, :, :, 0]

# If you want to use a local example for more realistic times, replace the imarr variable
# My local variable:
local_file = '/home/zcr545/YuccaData/yucca_raw_data/Task001_TestTask/imagesTr/TestTask_1000_000.nii.gz'
im = nib.load(local_file)
imarr = im.get_fdata()

# Convert it to the format expected of the transforms
# a dict of {"image": image, "seg": segmentation}
# with samples for dims (b, c, x, y, z) for 3D or (b, c, x, y) for 2D
imarr = imarr[np.newaxis]
seg = np.zeros(imarr.shape)
datadict = {"image": imarr, "seg": seg}
datadict = tio.Subject(
    t1=tio.ScalarImage(tensor=imarr),
    label=tio.LabelMap(tensor=seg))
print("Data is shaped :", imarr.shape)

/home/zcr545/miniconda3/envs/yuccat2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data is shaped : (1, 256, 287, 256)


In [2]:
idx = np.array(imarr.shape[1:])//4
imarr_half = imarr[:, idx[0]:-idx[0], idx[1]:-idx[1], idx[2]:-idx[2]]
seg_half = np.zeros(imarr_half.shape)
datadict_half = tio.Subject(
    t1=tio.ScalarImage(tensor=imarr_half),
    label=tio.LabelMap(tensor=seg_half))

In [53]:
# Applying Deformation
tform = tio.transforms.RandomElasticDeformation()
print("Deformation: ", timeit.timeit(lambda: tform(deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Deformation half: ", timeit.timeit(lambda: tform(deepcopy(datadict_half)), number=iterations)/iterations)

Deformation:  9.16136989100005 
  Deformation half:  1.3109800734996497


In [54]:
# Applying Ghosting
tform = tio.transforms.RandomGhosting(num_ghosts=3, axes=(0, 1), intensity=0.6)
print("Motion Ghosting: ", timeit.timeit(lambda: tform(deepcopy(datadict)), number=iterations)/iterations, "\n ",
      "Motion Ghosting half: ", timeit.timeit(lambda: tform(deepcopy(datadict_half)), number=iterations)/iterations)

Motion Ghosting:  0.5419647674998487 
  Motion Ghosting half:  0.07828107749992341


In [11]:
from monai.transforms import Rand3DElastic
datadict = {"image": imarr, "seg": seg}

# create an elsatic deformation transform
deform = Rand3DElastic(
    prob=1.0,
    sigma_range=(8., 8.),
    magnitude_range=(3., 3.))

print("Deform: ", timeit.timeit(lambda: deform(imarr[0]), number=iterations)/iterations)


KeyboardInterrupt: 

In [5]:
imarr_half.shape

(1, 128, 145, 128)